## 채용공고 scrapping

In [6]:
# pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver

In [187]:
browser = webdriver.Chrome('./chromedriver.exe')

In [188]:
browser.get('https://www.jobindexworld.com/jobpost/list')

In [189]:
html = browser.page_source

In [5]:
from bs4 import BeautifulSoup

In [190]:
soup = BeautifulSoup(html, 'html.parser')

### 채용공고 list를 클릭해서 들어가기

In [378]:
recruit_title = [] # 주제
company_name = [] # 회사명
detail_uri = [] # 내용 url

apply_start_date = [] # 모집기간
apply_end_date = []

task = [] # 직무개요
need_career = [] # 경력요건
team_env = [] # 처우
need_doc = [] # 제출서류

contact_people = [] # 담당컨설턴트
hashtag =[] # 해시태그

In [379]:
import time

for i in range(3):
    # click해서 들어간 창
    browser.find_elements_by_css_selector('article.cpn-circle-conts-item')[i].click() # 클릭 페이지
    html_scraping = browser.page_source 
    soup_scraping = BeautifulSoup(html_scraping, 'html.parser')
    
    recruit = soup_scraping.select('article.circle-conts-view > h3.tit-h3') # 주제, 제목
    recruit_t = recruit[0].text.strip()
    recruit_title.append(recruit_t)
    
    uri = soup_scraping.select('meta[property="og:url"]') # 구직 회사 내용 화면
    uri_t = uri[0]['content']
    detail_uri.append(uri_t)
    
    start_date = soup_scraping.select('div.date')
    start_date_t = start_date[0].text.strip()[1:9] # 날짜, 시간
    apply_start_date.append(start_date_t)
    
    tags = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div')
    
    tags_li = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div > ul > li')
    for i in range(len(tags)):
        if tags_li[i].text.strip()[8:] == '회사개요':
            task_t = tags_li[i+1].text.strip()
            task.append(task_t)
            # print(tags[i+1].text.strip())
        elif tags_li[i].text.strip()[-4:] == '경력레벨':
            need_career_t = tags_li[i+1].text.strip()
            need_career.append(need_career_t)
            # print(tags[i+1].text.strip())
        elif tags_li[i].text.strip()[7:] == '급여수준':
            team_env_t = tags_li[i+1].text.strip()
            team_env.append(team_env_t)
            # print(tags[i+1].text.strip())
        elif tags_li[i].text.strip()[7:] == '교육수준':
            need_doc_t = tags_li[i+1].text.strip()
            need_doc.append(need_doc_t)
        elif tags_li[i].text.strip()[8:] == '모집기간':
            apply_end_date_t = tags_li[i+1].text.strip()
            apply_end_date.append(apply_end_date_t)
            # print(tags[i+1].text.strip())

            # print(tags[i+1].text.strip())     
       
    for i in range(len(tags)):
        if tags[i].text.strip()[2:7] == '직무 개요':
            task_t = tags[i+1].text.strip()
            task.append(task_t)
            # print(tags[i+1].text.strip())
        elif tags[i].text.strip()[2:7] == '경력 요건':
            need_career_t = tags[i+1].text.strip()
            need_career.append(need_career_t)
            # print(tags[i+1].text.strip())
        elif tags[i].text.strip()[2:4] == '처우':
            team_env_t = tags[i+1].text.strip()
            team_env.append(team_env_t)
            # print(tags[i+1].text.strip())
        elif tags[i].text.strip()[2:7] == '제출 기한':
            apply_end_date_t = tags[i+1].text.strip()
            apply_end_date.append(apply_end_date_t)
            # print(tags[i+1].text.strip())
        elif tags[i].text.strip()[2:12] == '제출 서류 및 기타':
            need_doc_t = tags[i+1].text.strip()
            need_doc.append(need_doc_t)
            # print(tags[i+1].text.strip())        
        elif tags[i].text.strip()[2:9] == '담당 컨설턴트':
            contant_people_t = tags[i+1].text.strip()
            # print(tags[i+1].text.strip())

    contact = soup_scraping.select('span.writer-info')
    contact_t = contact[0].text.strip()[0:3]
    contact_people.append(contact_t)
    
    hash_col = []
    hash = soup_scraping.select('div.keyword-wrap > a.keyword') # 해시태그
    for j in range(len(hash)):  
        hash_t = hash[j].text.strip()
        hash_col.append(hash_t)
    hashtag.append(hash_col)
    
    time.sleep(1)
    browser.back()


In [380]:
recruit_title, detail_uri, contact_people, hashtag, apply_start_date

(['클라우드 인프라(네트워크)기술전문가(과-차장급)', 'Vision AI Engineer', '융합보안 컨설턴트'],
 ['https://www.jobindexworld.com/jobpost/view/53387',
  'https://www.jobindexworld.com/jobpost/view/53304',
  'https://www.jobindexworld.com/jobpost/view/53365'],
 ['최수현', '최수현', '최수현'],
 [['#IT', '#Cloud', '#infra', '#네트워크', '#브레인202_최수현헤드헌터'],
  ['#AI', '#IT', '#IT_Engineer', '#ComputerVision', '#브레인202_최수현헤드헌터'],
  ['#IT',
   '#보안',
   '#보안컨설팅',
   '#Cloud',
   '#AI',
   '#빅데이터',
   '#브레인202_최수현헤드헌터',
   '#DT',
   '#Pre-sales']],
 ['시간 전', '21.08.25', '21.09.02'])

In [381]:
apply_end_date, task, need_career, team_env, need_doc

([], [], [], [], [])

In [370]:
tags_li[3].text.strip()[7:]

' \n                                                            상관없음'

In [359]:
tags = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div')

In [372]:
# tags = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div')
tags_li = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div > ul > li')
for i in range(len(tags)):
    if tags_li[i].text.strip()[8:] == '회사개요':
        task_t = tags_li[i+1].text.strip()
        task.append(task_t)
        # print(tags[i+1].text.strip())
    elif tags_li[i].text.strip()[-4:] == '경력레벨':
        need_career_t = tags_li[i+1].text.strip()
        need_career.append(need_career_t)
        # print(tags[i+1].text.strip())
    elif tags_li[i].text.strip()[7:] == '급여수준':
        team_env_t = tags_li[i+1].text.strip()
        team_env.append(team_env_t)
        # print(tags[i+1].text.strip())
    elif tags_li[i].text.strip()[7:] == '교육수준':
        need_doc_t = tags_li[i+1].text.strip()
        need_doc.append(need_doc_t)
    elif tags_li[i].text.strip()[8:] == '모집기간':
        apply_end_date_t = tags_li[i+1].text.strip()
        apply_end_date.append(apply_end_date_t)
        # print(tags[i+1].text.strip())
    
        # print(tags[i+1].text.strip())       

In [235]:
browser.find_elements_by_css_selector('article.cpn-circle-conts-item')[4].click()

In [236]:
html_scraping = browser.page_source

In [237]:
soup_scraping = BeautifulSoup(html_scraping, 'html.parser')

In [238]:
start_date = soup_scraping.select('div.date')
start_date_t = start_date[0].text.strip()[1:9]

In [304]:
tags = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div')
# 회사개요 
tags[0], tags[0].ul.li.contents, tags[0].ul.li.text, tags[0].ul.li.strong.text
# 직무개요
tags[1], type(tags[1]), tags[1].contents, tags[1].strong.text, tags[2].text

(<div>◈ <strong>직무 개요</strong><br/></div>,
 bs4.element.Tag,
 ['◈ ', <strong>직무 개요</strong>, <br/>],
 '직무 개요',
 '○ 리모콘 개발')

In [154]:
tags[0].ul.li.text.strip()[:4]

'회사개요'

In [241]:
tags[0], tags[1], tags[2]

(<div class="jobpost-summary-wrapper">
 <ul>
 <li>
 <strong>회사개요 : </strong> 국내 대기업 계열 통신회사
                                                         </li>
 <li>
 <strong>모집기간 : </strong> 채용시까지
                                                         </li>
 </ul>
 </div>,
 <div>◈ <strong>직무 개요</strong><br/></div>,
 <div>○ 리모콘 개발</div>)

In [159]:
len(tags), type(tags)

(26, bs4.element.ResultSet)

In [274]:
tag_s = tag[0]
tags[1:len(tags)]

[<div>◈ <strong>직무 개요</strong><br/></div>,
 <div>○ 리모콘 개발</div>,
 <div><span style="font-family: inherit; font-size: 0.92857rem;">- 신규 리모콘 기획 및 개발</span><br/></div>,
 <div><br/></div>,
 <div>◈ <strong>경력 요건</strong><br/></div>,
 <div>- 학사 이상</div>,
 <div><span style="font-family: inherit; font-size: 0.92857rem;">○ 경력</span><br/></div>,
 <div>- <span style="font-family: inherit; font-size: 0.92857rem;">리모콘 개발 및 양산 경험</span></div>,
 <div><span style="font-family: inherit; font-size: 0.92857rem;">○ 역량/지식/자격증</span><br/></div>,
 <div>- <span style="font-family: inherit; font-size: 0.92857rem;">Bluetooth 기술 및 표준 규격에 대한 이해</span></div>,
 <div><span style="font-family: inherit; font-size: 0.92857rem;">○ 우대사항</span><br/></div>,
 <div>- <span style="font-family: inherit; font-size: 0.92857rem;">프로젝트 전체 PM 역할 경험 우대</span></div>,
 <div><span style="font-family: inherit; font-size: 0.92857rem;">- 미디어 </span><span style="font-family: inherit; font-size: 0.92857rem;">Device를 상용화 하는 제조사 출신 선호</span><

In [261]:
tags[0].text.strip()[2:7]

'직무 개요'

In [147]:
tags

[<div class="jobpost-summary-wrapper">
 <ul>
 <li>
 <strong>회사개요 : </strong> 국내 대기업 계열 통신회사
                                                         </li>
 <li>
 <strong>모집기간 : </strong> 채용시까지
                                                         </li>
 </ul>
 </div>,
 <div>◈ <strong>직무 개요</strong><br/></div>,
 <div>○ 리모콘 개발</div>,
 <div><span style="font-family: inherit; font-size: 0.92857rem;">- 신규 리모콘 기획 및 개발</span><br/></div>,
 <div><br/></div>,
 <div>◈ <strong>경력 요건</strong><br/></div>,
 <div>- 학사 이상</div>,
 <div><span style="font-family: inherit; font-size: 0.92857rem;">○ 경력</span><br/></div>,
 <div>- <span style="font-family: inherit; font-size: 0.92857rem;">리모콘 개발 및 양산 경험</span></div>,
 <div><span style="font-family: inherit; font-size: 0.92857rem;">○ 역량/지식/자격증</span><br/></div>,
 <div>- <span style="font-family: inherit; font-size: 0.92857rem;">Bluetooth 기술 및 표준 규격에 대한 이해</span></div>,
 <div><span style="font-family: inherit; font-size: 0.92857rem;">○ 우대사항</span><br/></div>

In [59]:
page = soup_scraping.select('div.circle-conts-view-main-conts-box-question')

In [60]:
page

[<div class="circle-conts-view-main-conts-box-question">
 <div class="jobpost-summary-wrapper">
 <ul>
 <li>
 <strong>회사개요 : </strong> 국내 유명 온라인 플랫폼 기업
                                                         </li>
 <li>
 <strong>모집기간 : </strong> 채용시까지
                                                         </li>
 </ul>
 </div>
 <div>◈ <strong>직무 개요</strong><br/></div>
 <div>- 커머스/파트너 플랫폼 UI/UX 디자인<br/>- 시장/고객/경쟁사/자사 분석을 통한 UX 전략 도출<br/></div>
 <div><br/></div>
 <div>◈ <strong>경력 요건</strong><br/></div>
 <div>- 이커머스 또는 플랫폼 Web/App 서비스의 UI/UX 디자인 경험 5년 이상<br/>- 시장과 고객에 대한 이해를 바탕으로 기획, 디자인이 가능하신 분<br/>- 비즈니스와 고객의 관점에서 UX 기획, 디자인이 가능하신 분<br/>- 디자인 원칙 중심으로 점진적 디자인 개선을 경험해보신 분<br/>- 디자인 툴 (Adobe, Figma, Protopie)의 툴 사용이 능숙하신 분<br/><br/></div>
 <div><span style="font-family: inherit; font-size: 0.92857rem;">◈ </span><strong style="font-family: inherit; font-size: 0.92857rem;">처우</strong><br/></div>
 <div>- 협의</div>
 <div><br/></div>
 <div>◈ <strong>제출 기한</strong><br/></div>
 <div>- ASAP</div>

In [37]:
soup_scraping.select('div.circle-conts-view-main-conts-box-question > div.jobpost-summary-wrapper > ul > li')

[<li>
 <strong>회사개요 : </strong> 글로벌 소비재 브랜드를 보유한 외국계 기업(한국 법인)
                                                         </li>,
 <li>
 <strong>경력레벨 : </strong>
                                                             mid-level, Senior
                                                         </li>,
 <li>
 <strong>급여수준 : </strong>협의
                                                         </li>,
 <li>
 <strong>교육수준 : </strong> 
                                                             학사, 석사/MBA
                                                         </li>,
 <li>
 <strong>모집기간 : </strong> 2021-09-19
                                                         </li>]

In [34]:
strong[0].text[0:4]

'회사개요'

In [36]:
if strong[0].text[0:4] == '회사개요':
    

In [40]:
soup_scraping.select('div.circle-conts-view-main-conts-box-question > p')

[<p><span style="color:#222222"><span style="font-size:11pt">[회사] 글로벌 브랜드를 보유한 외국계 소비재 대기업</span></span><br/>
 <span style="font-size:11pt"><span style="background-color:#ffffff"><span style="color:#222222">☞ 연봉 및 복리후생 국내 최고 수준으로 직원들의 근무 만족도가 높습니다.</span></span></span><br/>
 <br/>
 <span style="font-size:11pt"><span style="background-color:#ffffff"><span style="color:#222222">[포지션] 정규직<br/>
 - <strong>UX기획자(Customer Experience 관련)</strong></span></span></span><br/>
 <br/>
 <span style="font-size:11pt"><span style="background-color:#ffffff"><span style="color:#222222">[근무지]</span></span></span><br/>
 <span style="font-size:11pt"><span style="background-color:#ffffff"><span style="color:#222222">- 서울시 강남구</span></span></span><br/>
 <br/>
 <span style="font-size:11pt"><span style="background-color:#ffffff"><span style="color:#222222">[주요업무] *상세JD 별도전달 가능</span></span></span><br/>
 <span style="font-size:11pt"><span style="background-color:#ffffff"><span style="color:#222222">- <strong>자사의

### tag를 활용하여 select

In [59]:
recruit_title = soup.select('article.circle-conts-view > h3.tit-h3') # 주제, 제목
company_name = soup.select('div.jobpost-summary-wrapper > ul > li') # 회사명 -> 회사개요
detail_uri = soup.select('meta[property="og:url"]') # 구직 회사 내용 화면
apply_start_date = soup.select('') # 모집기한
apply_end_date = soup.select('')
task = soup.select('') # 직무개요 div.circle-conts-view-main-conts-box-question > div [3,4]
need_career = soup.select('') # 경력요건  div.circle-conts-view-main-conts-box-question > div [7]
team_env = soup.select('') # 처우  div.circle-conts-view-main-conts-box-question > div [10]
need_doc = soup.select('') # 제출서류  div.circle-conts-view-main-conts-box-question > div []
contact_people = soup.select('') # 담당컨설턴트  div.circle-conts-view-main-conts-box-question > div
hashtag = soup.select('div.keyword-wrap') # 해시태그

SelectorSyntaxError: Expected a selector at position 0
  line 1:

^

In [71]:
recruit_title = soup.select('article.circle-conts-view > h3.tit-h3') # 주제, 제목

In [161]:
len(recruit_title), type(recruit_title), print(recruit_title)

[[], [], []]


(3, list, None)

In [76]:
company_name = soup.select('div.jobpost-summary-wrapper > ul > li') # 회사명

In [77]:
print(company_name)

[<li>
<strong>회사개요 : </strong> 국내 대기업 계열 시큐리티 솔루션 기업
                                                        </li>, <li>
<strong>모집기간 : </strong> 채용시까지
                                                        </li>]


In [78]:
detail_uri = soup.select('meta[property="og:url"]') # 구직 회사 내용 화면

In [86]:
print(detail_uri)

[<meta content="https://www.jobindexworld.com/jobpost/view/53346" property="og:url"/>]


In [88]:
detail_uri[0].text

''

In [73]:
hashtag = soup.select('div.keyword-wrap') # 해시태그

In [74]:
len(hashtag), type(hashtag)

(1, bs4.element.ResultSet)

In [75]:
print(hashtag)

[<div class="keyword-wrap">
<a class="keyword" href="/circle/tags/view/3011">#Data</a><input name="keyword-index-ref" type="hidden" value="3011"/><a class="keyword" href="/circle/tags/view/5016">#데이터사이언티스트</a><input name="keyword-index-ref" type="hidden" value="5016"/><a class="keyword" href="/circle/tags/view/4259">#DATA_SCIENTIST</a><input name="keyword-index-ref" type="hidden" value="4259"/>
</div>]


In [83]:
hashtag[0].text

'\n#Data#데이터사이언티스트#DATA_SCIENTIST\n'

In [81]:
contents = list()
for hash in hashtag:
    h_tag = hash.select('div.keyword-wrap')
    contents.append(h_tag)

contents

[[]]

### SCRAPING SITE TABLE에 넣기

In [58]:
brawser = webdriver.Chrome('./chromedriver.exe')

In [59]:
brawser.get('https://github.com/login')

In [60]:
brawser.find_elements_by_css_selector('input#login_field')[0].send_keys('superinssa@gmail.com')

In [63]:
brawser.find_elements_by_css_selector('input#password')[0].send_keys('ghlehd123#')

In [64]:
brawser.find_elements_by_css_selector('input[type="submit"]')[0].click()